# YoloV5 Training at GPU -- Applying Gaussian Filter at Ignored Areas

Treinamento de uma rede **YOLOv5-S** nos dados do UAVDT Dataset. As imagens com zona de "ignore" são tratadas; nesses boxes, é aplicado um filtro gaussiano para "borrar" a imagem de forma a não deixar nenhum contorno aparente.

Autor: Yuri Gonçalves Ribeiro

Data: 08 de Outubro de 2020

Exemplo de Execução:

```bash
conda activate yv5; experimento="YOLOv5_UAVDT_2_TRAIN"; outputFileName=$(date "+"$experimento"_ipynb_%d_%B_%Y_%Hh_%Mm_%Ss"); echo "Salvando log de execucao em: $outputFileName.txt"; nohup jupyter nbconvert --to notebook --execute --allow-errors --output "./logs/$outputFileName" --ExecutePreprocessor.timeout=-1 --Application.log_level=10 $experimento.ipynb &> "./logs/$outputFileName.txt" &
```

Os resultados são salvos na pasta do Dropbox especificada abaixo.

# Etiqueta do Experimento (Yolov5 Commit Hash, Conda Env, Informações da Máquina)

## Commit Hash da Rede YoloV5 utilizada neste experimento e pacotes instalados no conda environment

In [1]:
%%script bash
echo "Hash key do commit YoloV5 que esta em uso:"
cd ../../Submodules/yolov5
echo $( git rev-parse --verify HEAD)
echo
echo "Verificar localizacao do python sendo executado:"
which python
echo
echo "Conda Environment Utilizado: "
conda env export

Hash key do commit YoloV5 que esta em uso:
b42e8a531b8eb12b2f71c00ddc4dea187633f7c7

Verificar localizacao do python sendo executado:
/home/yuri/anaconda3/envs/yv5/bin/python

Conda Environment Utilizado: 
name: yv5
channels:
  - conda-forge
  - defaults
dependencies:
  - _libgcc_mutex=0.1=main
  - argon2-cffi=20.1.0=py38h1e0a361_1
  - attrs=19.3.0=py_0
  - backcall=0.2.0=pyh9f0ad1d_0
  - backports=1.0=py_2
  - backports.functools_lru_cache=1.6.1=py_0
  - bleach=3.1.5=pyh9f0ad1d_0
  - ca-certificates=2020.6.20=hecda079_0
  - certifi=2020.6.20=py38h32f6830_0
  - cffi=1.14.1=py38he30daa8_0
  - decorator=4.4.2=py_0
  - defusedxml=0.6.0=py_0
  - entrypoints=0.3=py38h32f6830_1001
  - importlib-metadata=1.7.0=py38h32f6830_0
  - importlib_metadata=1.7.0=0
  - ipykernel=5.3.4=py38h23f93f0_0
  - ipython=7.17.0=py38h1cdfbd6_0
  - ipython_genutils=0.2.0=py_1
  - jedi=0.15.2=py38_0
  - jinja2=2.11.2=pyh9f0ad1d_0
  - jsonschema=3.2.0=py38h32f6830_1
  - jupyter_client=6.1.6=py_0
  - jupyter_core=4.6

## Especificações do Computador e do Experimento

In [2]:
# Especificações do computador, para saber onde estou rodando o código...
import datetime
from platform import uname
from os import environ
from glob import glob
import cv2

start_time = datetime.datetime.now()
time_stamp = start_time.strftime("%d_%B_%Y_%Hh_%Mm_%Ss")

username = environ['USER']
sysinfo = uname()
hostname = sysinfo.node

root = "~/Desktop/Mestrado/Codigos_Dissertacao/RedesNeurais/YOLOv5_UAVDT_2"

dropbox_dir = "~/Dropbox"

experimento = "YOLOv5_UAVDT_2"

output_dir = "%s/YoloV5_Runs/Experimentos/%s_%s"%(dropbox_dir,experimento,time_stamp)


# Checar conda environment
conda_env = !printenv CONDA_PROMPT_MODIFIER
if conda_env != ['(yv5) ']:
    raise SystemExit("ERRO.. Conda environment incorreto.")

In [3]:
print("""
==Computer Specifications==
username: {username}

hostname: {hostname}

system info: {sysinfo}

{{root}}: {root}

dropbox dir: {dropbox_dir}

experimento: {experimento}

time stamp: {time_stamp}

output dir: {output_dir}
===========================
""".format(username = username, hostname = hostname, sysinfo = sysinfo,
           conda_env = conda_env, root = root, dropbox_dir = dropbox_dir,
           time_stamp = time_stamp, experimento = experimento, output_dir = output_dir))


==Computer Specifications==
username: yuri

hostname: febe

system info: uname_result(system='Linux', node='febe', release='4.15.0-117-generic', version='#118~16.04.1-Ubuntu SMP Sat Sep 5 23:35:06 UTC 2020', machine='x86_64', processor='x86_64')

{root}: ~/Desktop/Mestrado/Codigos_Dissertacao/RedesNeurais/YOLOv5_UAVDT_2

dropbox dir: ~/Dropbox

experimento: YOLOv5_UAVDT_2

time stamp: 13_October_2020_14h_48m_28s

output dir: ~/Dropbox/YoloV5_Runs/Experimentos/YOLOv5_UAVDT_2_13_October_2020_14h_48m_28s



In [4]:
!nvidia-smi

Tue Oct 13 14:48:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   51C    P0    58W / 280W |      0MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 90%   85C    P2   187W / 280W |   9413MiB / 11178MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

# Clonar o Repositório YOLOv5 e instalar os requirements.txt

In [5]:
#Ir para a raiz
%cd {root}
!echo
# Remover diretórios antigos
!rm -rf runs yolov5
#Criar pasta de runs
!mkdir runs
!echo
#Copiar diretório novo da pasta submodules
!cp -av ../../Submodules/yolov5/ yolov5/
!echo
!ls
#Entrar na pasta yolov5 e instalar os requirements.txt
%cd {root}/yolov5/
%pip install -U -r requirements.txt

/home/yuri/Desktop/Mestrado/Codigos_Dissertacao/RedesNeurais/YOLOv5_UAVDT_2


'../../Submodules/yolov5/' -> 'yolov5/'
'../../Submodules/yolov5/Dockerfile' -> 'yolov5/Dockerfile'
'../../Submodules/yolov5/README.md' -> 'yolov5/README.md'
'../../Submodules/yolov5/detect.py' -> 'yolov5/detect.py'
'../../Submodules/yolov5/hubconf.py' -> 'yolov5/hubconf.py'
'../../Submodules/yolov5/.git' -> 'yolov5/.git'
'../../Submodules/yolov5/.dockerignore' -> 'yolov5/.dockerignore'
'../../Submodules/yolov5/.gitattributes' -> 'yolov5/.gitattributes'
'../../Submodules/yolov5/.gitignore' -> 'yolov5/.gitignore'
'../../Submodules/yolov5/requirements.txt' -> 'yolov5/requirements.txt'
'../../Submodules/yolov5/LICENSE' -> 'yolov5/LICENSE'
'../../Submodules/yolov5/test.py' -> 'yolov5/test.py'
'../../Submodules/yolov5/train.py' -> 'yolov5/train.py'
'../../Submodules/yolov5/tutorial.ipynb' -> 'yolov5/tutorial.ipynb'
'../../Submodules/yolov5/.github' -> 'yolov5/.github'
'../../Submodules/yolov5/.github/ISSUE_TEMPLAT

# Executar Script para Transformar Imagens do UAVDT Dataset para o formato requerido pelo YOLOv5

In [6]:
# Ir para a raiz
%cd {root}

/home/yuri/Desktop/Mestrado/Codigos_Dissertacao/RedesNeurais/YOLOv5_UAVDT_2


In [7]:
%%script bash
# Apagar os links existentes para o dataset
rm -rfv UAVDT_YOLOv5
# Criar novos hard links para o dataset e organizer o dataset no formato exigido pelo yolov5
mkdir UAVDT_YOLOv5
# Executar o script para converter UAVDT para o formato YOLOv5:
cd ../../misc

python uavdt_to_yolov5.py --conjunto treinoEteste --dest ../RedesNeurais/YOLOv5_UAVDT_2/UAVDT_YOLOv5 --nosymlink

cd ../RedesNeurais/YOLOv5_UAVDT_2/

[INFO] Etiqueta de Tempo: 13_10_2020_14_48_34.
[INFO] Iniciando processo de transformação para Treino.
[INFO] Analisando imagens do clipe: M0101...
[INFO] Analisando imagens do clipe: M0201...
[INFO] Analisando imagens do clipe: M0202...
[INFO] Analisando imagens do clipe: M0204...
[INFO] Analisando imagens do clipe: M0206...
[INFO] Analisando imagens do clipe: M0207...
[INFO] Analisando imagens do clipe: M0210...
[INFO] Analisando imagens do clipe: M0301...
[INFO] Analisando imagens do clipe: M0401...
[INFO] Analisando imagens do clipe: M0402...
[INFO] Analisando imagens do clipe: M0501...
[INFO] Analisando imagens do clipe: M0603...
[INFO] Analisando imagens do clipe: M0604...
[INFO] Analisando imagens do clipe: M0605...
[INFO] Analisando imagens do clipe: M0702...
[INFO] Analisando imagens do clipe: M0703...
[INFO] Analisando imagens do clipe: M0704...
[INFO] Analisando imagens do clipe: M0901...
[INFO] Analisando imagens do clipe: M0902...
[INFO] Analisando imagens do clipe: M1002.

# Preprocessar as imagens
## Salvá-las temporariamente (enquanto estiver treinando a rede). Após o término do treinamento, deve-se apagar as imagens objetivando economizar espaço em disco.

In [8]:
movies_treino = ["M0101", "M0201", "M0202", "M0204", \
    "M0206", "M0207", "M0210", "M0301", "M0401", "M0402", \
    "M0501", "M0603","M0604", "M0605", "M0702", "M0703", \
    "M0704", "M0901", "M0902", "M1002", "M1003", "M1005", \
    "M1006", "M1008", "M1102", "M1201", "M1202", "M1304", \
    "M1305", "M1306"]

for movie_name in movies_treino:
    print(f"[INFO] Analisando a sequência de imagens {movie_name}.")

    frame_boxes = {}
    with open(f"../../Datasets/UAVDT/UAV-benchmark-MOTD_v1.0/GT/{movie_name}_gt_ignore.txt") as igfile:
        lines = igfile.readlines()
        for line in lines:
            line = line.split(",")
            frame = int(line[0])
            x = int(line[2])
            y = int(line[3])
            w = int(line[4])
            h = int(line[5])
            classe = int(line[-1])
            if frame in frame_boxes:
                frame_boxes[frame] += [[frame, x, y, w, h, classe]]
            else:
                frame_boxes[frame] = [[frame, x, y, w, h, classe]]

    # Para cada vídeo, ajustar as imagens do UAVDT Local com os devidos blurs..
    for frame, boxes in frame_boxes.items():
        img_path = f"./UAVDT_YOLOv5/train/images/{movie_name}_img{frame:06}.jpg"
        img = cv2.imread(img_path)
        for _,x,y,w,h,_ in boxes:
            # Kernel Sized
            kw = w//2
            kh = h//2
            if kw % 2 == 0: kw -= 1
            if kh % 2 == 0: kh -= 1
            # Gaussian Blur
            roi = img[y:y+h, x:x+w]
            roi_blur = cv2.GaussianBlur(roi, (kw, kh), 20, 20, cv2.BORDER_REPLICATE)
            img[y:y+h, x:x+w] = roi_blur
        
        cv2.imwrite(img_path, img)

"""
# Checar efeitos do Gaussian Blur em uma "ignore area" de uma imagem qualquer do dataset.

img = cv2.imread("../../Datasets/UAVDT/UAV-benchmark-M/M0101/img000001.jpg")
# Boxes da imagem 000001.jpg do filme M0101; [frame, x, y, w, h, classe]
frame1_boxes_img000001_M0101 = [[1, 914, 189, 105, 54, -1],  [1, 956, 480, 64, 58, -1],  [1, 788, 291, 121, 113, -1]]

for _,x,y,w,h,_ in frame1_boxes_img000001_M0101:
    # Kernel Sized
    kw = w//2
    kh = h//2
    if kw % 2 == 0: kw -= 1
    if kh % 2 == 0: kh -= 1
    # 2xGaussian Blur
    roi = img[y:y+h, x:x+w]
    roi_blur = cv2.GaussianBlur(roi, (kw, kh), 20, 20, cv2.BORDER_REPLICATE)
    img[y:y+h, x:x+w] = roi_blur

cv2.imshow("Teste", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""
pass

[INFO] Analisando a sequência de imagens M0101.
[INFO] Analisando a sequência de imagens M0201.
[INFO] Analisando a sequência de imagens M0202.
[INFO] Analisando a sequência de imagens M0204.
[INFO] Analisando a sequência de imagens M0206.
[INFO] Analisando a sequência de imagens M0207.
[INFO] Analisando a sequência de imagens M0210.
[INFO] Analisando a sequência de imagens M0301.
[INFO] Analisando a sequência de imagens M0401.
[INFO] Analisando a sequência de imagens M0402.
[INFO] Analisando a sequência de imagens M0501.
[INFO] Analisando a sequência de imagens M0603.
[INFO] Analisando a sequência de imagens M0604.
[INFO] Analisando a sequência de imagens M0605.
[INFO] Analisando a sequência de imagens M0702.
[INFO] Analisando a sequência de imagens M0703.
[INFO] Analisando a sequência de imagens M0704.
[INFO] Analisando a sequência de imagens M0901.
[INFO] Analisando a sequência de imagens M0902.
[INFO] Analisando a sequência de imagens M1002.
[INFO] Analisando a sequência de imagens

# Criar o arquivo data.yaml

In [9]:
# O conteúdo original não está adequado... mais fácil que editar: removerei-o e criarei um novo.
%cd {root}

datayaml = """# train and val datasets (image directory or *.txt file with image paths)

train: ../UAVDT_YOLOv5/train/images/
val: ../UAVDT_YOLOv5/test/images/

# number of classes
nc: 3

# class names
names: ['car', 'truck', 'bus']
"""

%rm data.yaml
with open('data.yaml', 'w') as file:
    file.write(datayaml)
%cat data.yaml

/home/yuri/Desktop/Mestrado/Codigos_Dissertacao/RedesNeurais/YOLOv5_UAVDT_2
rm: cannot remove 'data.yaml': No such file or directory
# train and val datasets (image directory or *.txt file with image paths)

train: ../UAVDT_YOLOv5/train/images/
val: ../UAVDT_YOLOv5/test/images/

# number of classes
nc: 3

# class names
names: ['car', 'truck', 'bus']


# Iniciar o Treinamento por 300 eṕocas


In [10]:
%cd {root}/yolov5
epocas = 300

!python train.py --epochs {epocas} --cfg ./models/yolov5s.yaml --weights ../../../Weights/YoloV5/yolov5s.pt  --data ../data.yaml --img 640 --batch-size 16 --device 0 --logdir ../runs --workers 4


/home/yuri/Desktop/Mestrado/Codigos_Dissertacao/RedesNeurais/YOLOv5_UAVDT_2/yolov5
Using CUDA device0 _CudaDeviceProperties(name='GeForce GTX 1080 Ti', total_memory=11178MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='./models/yolov5s.yaml', data='../data.yaml', device='0', epochs=300, evolve=False, global_rank=-1, hyp='data/hyp.finetune.yaml', img_size=[640, 640], local_rank=-1, logdir='../runs', multi_scale=False, name='', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, sync_bn=False, total_batch_size=16, weights='../../../Weights/YoloV5/yolov5s.pt', workers=4, world_size=1)
Start Tensorboard with "tensorboard --logdir ../runs", view at http://localhost:6006/
Hyperparameters {'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'transl

# Guardar os Resultados na Nuvem (Dropbox)

In [11]:
%cd {root}
%mkdir -p {output_dir}
%cp -av logs/ {output_dir}
%cp -av runs/ {output_dir}

/home/yuri/Desktop/Mestrado/Codigos_Dissertacao/RedesNeurais/YOLOv5_UAVDT_2
'logs/' -> '/home/yuri/Dropbox/YoloV5_Runs/Experimentos/YOLOv5_UAVDT_2_13_October_2020_14h_48m_28s/logs'
'logs/YOLOv5_UAVDT_2_TRAIN_ipynb_13_October_2020_14h_48m_24s.txt' -> '/home/yuri/Dropbox/YoloV5_Runs/Experimentos/YOLOv5_UAVDT_2_13_October_2020_14h_48m_28s/logs/YOLOv5_UAVDT_2_TRAIN_ipynb_13_October_2020_14h_48m_24s.txt'
'logs/.ipynb_checkpoints' -> '/home/yuri/Dropbox/YoloV5_Runs/Experimentos/YOLOv5_UAVDT_2_13_October_2020_14h_48m_28s/logs/.ipynb_checkpoints'
'runs/' -> '/home/yuri/Dropbox/YoloV5_Runs/Experimentos/YOLOv5_UAVDT_2_13_October_2020_14h_48m_28s/runs'
'runs/exp0' -> '/home/yuri/Dropbox/YoloV5_Runs/Experimentos/YOLOv5_UAVDT_2_13_October_2020_14h_48m_28s/runs/exp0'
'runs/exp0/events.out.tfevents.1602612398.febe.21509.0' -> '/home/yuri/Dropbox/YoloV5_Runs/Experimentos/YOLOv5_UAVDT_2_13_October_2020_14h_48m_28s/runs/exp0/events.out.tfevents.1602612398.febe.21509.0'
'runs/exp0/hyp.yaml' -> '/home/yur

# Finalização

In [12]:
finish_time = datetime.datetime.now()
delta_time = finish_time - start_time
print("""
Start: {start}

Finish: {finish}

Duration (hour:min:sec.microsec):  {delta}
""".format(start = start_time, finish = finish_time, delta = delta_time))


Start: 2020-10-13 14:48:28.071410

Finish: 2020-10-16 10:55:34.652258

Duration (hour:min:sec.microsec):  2 days, 20:07:06.580848



In [13]:
# Manualmente, remover os logs e diretórios com imagens desnecessários.